# Backfill the data dir with supplemental vae images

In [1]:
import os
import shutil
from glob import glob
import pandas as pd
import numpy as np

np.random.seed(123)

In [2]:
data_dir = './dcai_gcb_01/dcai_gcb_01'
train_dir = data_dir + '/train'
val_dir = data_dir + '/val'
numerals = ["i", "ii", "iii", "iv", "v", "vi", "vii", "viii", "ix", "x"]

In [3]:
vae_frac = 0.2
target_counts = {
    'i': 492,
    'ii': 600,
    'iii': 700,
    'iv': 1500,
    'v': 800,
    'vi': 900,
    'vii': 1500,
    'viii': 1500,
    'ix': 1500,
    'x': 500,
}
# print(sum(target_counts.values()))
# target_counts = {k: int((1-vae_frac)*v) for k, v in target_counts.items()}
print(sum(target_counts.values()))

9992


In [4]:
def get_file_count(numeral):
    num_pattern = data_dir + '/*/{}/*.png'.format(numeral)
    fps = glob(num_pattern)
    return len(fps)

def get_n_required(numeral):
    n_files = get_file_count(numeral)
    n_required = target_counts[numeral] - n_files
    return n_required

def count_all_files():
    n_tot = 0
    for n in numerals:
        n_files = get_file_count(n)
        n_tot += n_files
        print('Numeral "{}" has {} files'.format(n, n_files))
    print('Total of {} files'.format(n_tot))

In [5]:
for n in numerals:
    vae_df = pd.read_csv('./vae_labels_{}.csv'.format(n), index_col=0)
    vae_df = vae_df[vae_df.good.astype(bool)]
    n_train_dir = train_dir + '/{}'.format(n)

    n_vae_files = get_n_required(n)

    vae_df = vae_df.sample(n=n_vae_files, replace=False)
    assert len(vae_df) == n_vae_files
    assert (vae_df['label'] == 'good').all()
    
    for fp in vae_df['fp']:
        dest = os.path.join(n_train_dir, os.path.basename(fp))
        shutil.copy(fp, dest)

In [6]:
count_all_files()

Numeral "i" has 492 files
Numeral "ii" has 600 files
Numeral "iii" has 700 files
Numeral "iv" has 1500 files
Numeral "v" has 800 files
Numeral "vi" has 900 files
Numeral "vii" has 1500 files
Numeral "viii" has 1500 files
Numeral "ix" has 1500 files
Numeral "x" has 500 files
Total of 9992 files
